In [ ]:
import itertools
import os 
import sys
import re
import math
import random
import requests
import seaborn as sns
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import ListedColormap

import matplotlib.pylab as plt
import numpy as np
import PIL.Image as Image
import pandas as pd
import tensorflow as tf
import keras
from tensorflow.keras.layers import Input
from keras.models import Sequential, Model
from keras.layers import BatchNormalization, Flatten, Input, Dense,Dropout,Activation,Flatten,Reshape,Lambda,Embedding,Conv2D,MaxPooling2D, UpSampling2D   #卷积层，池化层
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam

from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import tensorflow.python.platform.build_info as build 
import tensorflow_hub as hub 
import tensorflow_datasets as tfds 
from tensorflow.python.client import device_lib

%cd C:\Results_2m_0.56m_0.0683m

In [ ]:
%%time

def getTemp(starting_frame, ending_frame, info_current_dataset, img_current_dataset):
  for frame in range(starting_frame, ending_frame + 1):
    temp_filename = "TEMPERATURE_at_frame%d.txt" % frame 
    time_stamp_raw = pd.read_csv(temp_filename, sep='\t', header=None, 
                                skiprows=[i for i in range(0,4)], on_bad_lines='skip')
    timestamp = [float(s) for s in re.findall(r':\s(.*)', time_stamp_raw.iat[2, 1])]
    timestamp = timestamp[0]    
      
    temp_raw = pd.read_csv(temp_filename, header=None, sep='\t', 
                          skiprows=[i for i in range(0,9)]).iloc[[i for i in range(0, 17583 - 9)], [0, 1]]
    temp_raw.columns = ['node_label', 'temperature']
    temp = pd.merge(temp_raw, node_info, on = 'node_label')
    temp['step'] = frame
    temp['temperature'] = temp['temperature'] - 273.15

    if (frame == starting_frame):
      initial_time = timestamp
      x_dim = (temp['node_x'].unique()).size
      y_dim = (temp['node_y'].unique()).size
      z_dim = (temp['node_z'].unique()).size

    temp['time'] = (timestamp - initial_time)/60   
    temp['scaled_x'] = ( temp['node_x'] / max(temp['node_x']) ) * x_dim
    temp['scaled_y'] = ( (temp['node_y']) / (max(temp['node_y'])) ) * y_dim
    temp['scaled_z'] = ( (temp['node_z']) / (max(temp['node_z'])) ) * z_dim

    # cast original coordinates values into integers
    temp['X'] = temp.scaled_x.astype('int')
    temp['Y'] = temp.scaled_y.astype('int')
    temp['Z'] = temp.scaled_z.astype('int')
    info_current_dataset.append(temp)                     # append current frame
    
    # cast temperature data into images
    img_x = max(temp['X']) + 1   # 102 
    img_y = max(temp['Y']) + 1   # 30 
    img_z = max(temp['Z']) + 1   # 6  

    get_image = np.zeros(shape = (img_y, img_x, img_z))

    for i in range(len(temp)):
      x_coordinate = temp.loc[i, "X"]
      y_coordinate = temp.loc[i, "Y"]
      z_coordinate = temp.loc[i, "Z"]
      get_image[y_coordinate][x_coordinate][z_coordinate] = temp.loc[i, "temperature"]
    img_current_dataset.append(get_image)                 # append current frame
 
def getImgChannel(channel, dataset):
  output_img = []
  length = len(dataset)
  for i in range(length):
    output_img.append(dataset[i, :, :, channel])
  return np.array(output_img)

def get_all_ImgChannel(dataset):
  output_img = []
  length = len(dataset)
  for i in range(length):
    output_img.append(dataset[i, :, :, 0])
  for i in range(length):
    output_img.append(dataset[i, :, :, 1])
  for i in range(length):
    output_img.append(dataset[i, :, :, 2])
  for i in range(length):
    output_img.append(dataset[i, :, :, 3])
  for i in range(length):
    output_img.append(dataset[i, :, :, 4])
  for i in range(length):
    output_img.append(dataset[i, :, :, 6])
  return np.array(output_img)

def patch_image(dataset):
  for img in range(len(dataset)):
    for col in range(102):
      (dataset[img])[28, col] = ((dataset[img])[27, col] + (dataset[img])[29, col]) / 2
    for row in range(30):
      (dataset[img])[row, 100] = ((dataset[img])[row, 99] + (dataset[img])[row, 101]) / 2
    (dataset[img])[28, 100] = ((dataset[img])[27, 99] + (dataset[img])[27, 101] + (dataset[img])[29, 99] + (dataset[img])[29, 101]) / 4

################################################################################
node_info = pd.read_csv('Mesh_info.txt', delim_whitespace=True, header=None)
node_info.columns = ['node_label', 'node_x', 'node_y', 'node_z']
node_info = node_info.astype(str).astype(float)

temperature_info_all = [] 
temperature_img_all = []

getTemp(1, 541, temperature_info_all, temperature_img_all)

temperature_img_all_array = np.array(temperature_img_all)

layer_z6 = getImgChannel(6, temperature_img_all_array)
patch_image(layer_z6)
layer_z6_norm = layer_z6 / np.max(layer_z6)

images_all_layers = get_all_ImgChannel(temperature_img_all_array)
patch_image(images_all_layers)
images_all_layers_norm = images_all_layers / np.max(images_all_layers)

In [ ]:
######################### CNN architecture ##########################
 
img_size = Input(shape = (30, 102, 1), name = 'Input') 

x = Conv2D(16, (3, 3), strides = (1, 4),activation = 'relu', padding = 'same', name = 'enc_1_conv')(img_size)
x = MaxPooling2D((2, 2), padding = 'same', name = 'enc_2')(x)
x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'enc_3_conv')(x)
x = MaxPooling2D((2, 2), padding = 'same', name = 'enc_4')(x)
x = Conv2D(64, (3, 3), activation = 'relu', name = 'enc_5_conv')(x)
x = MaxPooling2D((2, 2), padding = 'same', name = 'enc_6')(x)
x = Conv2D(128, (3, 3), activation = 'relu', name = 'enc_7_conv')(x)
encoded = Flatten(name = 'enc_8')(x)

########################  Decoder ########################

x = Reshape((1, 1, 128), name = 'dec_1')(encoded)
x = UpSampling2D((2, 2), name = 'dec_2')(x)
x = Conv2D(64, (3, 3), activation = 'relu', padding = 'same', name = 'dec_3_conv')(x)
x = UpSampling2D((2, 2), name = 'dec_4')(x)
x = Conv2D(32, (3, 3), activation = 'relu', padding = 'same', name = 'dec_5_conv')(x)
x = UpSampling2D((3, 3), name = 'dec_6')(x)
x = Conv2D(16, (3, 3), activation = 'relu', name = 'dec_7_conv')(x)
x = UpSampling2D((3, 3), name = 'dec_8')(x)
x = Conv2D(16, (1, 3), activation = 'relu', name = 'dec_9_conv')(x)
x = Conv2D(16, (1, 3), activation = 'relu', name = 'dec_10_conv')(x)
x = UpSampling2D((1, 4), name = 'dec_11')(x)
decoded = Conv2D(1, (1, 3), activation = 'relu', name = 'dec_12_conv')(x)

autoencoder = Model(img_size, decoded)
autoencoder.summary()

In [ ]:
%%time

train_all, test_dataset = train_test_split(images_all_layers_norm, test_size = 0.2, random_state = 42)
train, validation = train_test_split(train_all, test_size = 0.2, random_state = 42)

train = train.reshape(train.shape[0], 30, 102, 1)
validation = validation.reshape(validation.shape[0], 30, 102, 1)
test = test_dataset.reshape(test_dataset.shape[0], 30, 102, 1)

epoch = 1500;
callbacks = [EarlyStopping(monitor='val_loss', patience=100, verbose=2),
             ModelCheckpoint('CNN-test.h5', monitor = 'val_loss', save_best_only = True, verbose=0)]

keras.backend.set_epsilon(1)
autoencoder.compile(loss='mae', optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
                    metrics=['mape'])   

history = autoencoder.fit(train, train, batch_size = 128, epochs = epoch, verbose = 1, 
                          validation_data = (validation, validation), workers=8, 
                          use_multiprocessing = 1, callbacks = callbacks) 

plt.plot(history.history['mape'], label = 'Training data')
plt.plot(history.history['val_mape'], label ='Validation data')
plt.ylabel('MAPE')
plt.xlabel('No. epoch')
plt.legend(loc="upper right")
plt.show()

In [ ]:
model_testing = autoencoder.evaluate(test, test, verbose = 2)
model_testing